# Transfer Learning with TensorFlow Part 2: Fine-tuning

In the previous notebook, we covered transfer learning feature extraction, now it's time to learn about a new kind of transfer-learning: fine-tuning

## Creating helper functions

In previous notebooks, we have created a bunch of helper functions, now we could rewrite them all, however, this is tedious.

So, it's a good idea to put functions you will want to use again in a script you can download and import into your notebooks (or elsewhere).

We have done this for some of the functions we have used previously.

In [1]:
import sys
import os

# Subir dos niveles: desde notebooks/pruebas → mi_proyecto/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# Ahora puedes importar
from utils.helper_functions import create_tensorboard_callback, plot_loss_curves, walk_through_dir

## Let's get some data

This time we are going to see how we can use the pretrained models within `tf.keras.applications` and apply them to our own problem (recognizing images of food).

In [2]:
# Get 10% of training data of 10% of Food101 and check out how many images and subdirectories are in our dataset
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [3]:
# Create training and test directory paths
train_dir = '10_food_classes_10_percent/train'
test_dir = "10_food_classes_10_percent/test"

In [4]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode='categorical',
                                                                            batch_size=BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                image_size=IMG_SIZE,
                                                                label_mode='categorical',
                                                                batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [5]:
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [6]:
# Check out class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [7]:
# See and example of a batch of data
for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[3.08979588e+01 2.45867348e+01 1.63571434e+01]
   [3.00255108e+01 2.30255108e+01 1.50255098e+01]
   [3.14948978e+01 2.24948978e+01 1.34948978e+01]
   ...
   [4.15304565e+00 7.65228271e-02 0.00000000e+00]
   [3.85711670e+00 7.14416504e-02 0.00000000e+00]
   [2.58675718e+00 1.58675718e+00 0.00000000e+00]]

  [[3.19285717e+01 2.49285717e+01 1.69285717e+01]
   [3.09948978e+01 2.38622456e+01 1.59285717e+01]
   [3.11428585e+01 2.21275520e+01 1.31275511e+01]
   ...
   [6.00000000e+00 1.00000000e+00 0.00000000e+00]
   [4.13776731e+00 1.37767226e-01 0.00000000e+00]
   [3.16836977e+00 1.95408416e+00 0.00000000e+00]]

  [[2.99336739e+01 2.19234695e+01 1.44285717e+01]
   [3.01989803e+01 2.12295914e+01 1.42142859e+01]
   [2.98775501e+01 2.00459175e+01 1.10459185e+01]
   ...
   [7.12241459e+00 1.61736393e+00 0.00000000e+00]
   [5.78571415e+00 1.05613267e+00 0.00000000e+00]
   [5.86225224e+00 1.86225224e+00 0.00000000e+00]]

  ...

  [[8.00000000e+00 8.00000000e+00 6.42858887e+00]
   [8

## Model 0: Building a transfer learning feature extraction model using the Keras Functional API

The sequential API is straight-forward, it runs our layers in sequential order.

But the functional API gives us more flexibility with our models.

In [9]:
# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)

# 2. Freeze the base model
base_model.trainable = False

# 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')

# 4. If using a model like ResNet50V2 you will need to normalize inputs
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. Pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. Average pool outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

# 10. Fit the model and save its history
history_10_percent = model_0.fit(train_data_10_percent,
                                 epochs=5,
                                 steps_per_epoch=len(train_data_10_percent),
                                 validation_data=test_data,
                                 validation_steps=int(0.25 * len(test_data)), # validate on a quarter
                                 callbacks=create_tensorboard_callback(dir_name='transfer_learning',
                                                                       experiment_name='10_percent_feature_extraction'))

Shape after passing inputs through base model: (None, 7, 7, 1280)
Shape after GlobalAveragePooling2D: (None, 1280)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extraction/20250722-222744
Epoch 1/5
24/24 [==============================] - 17s 142ms/step - loss: 1.8639 - accuracy: 0.3840 - val_loss: 1.2969 - val_accuracy: 0.7385
Epoch 2/5
24/24 [==============================] - 2s 82ms/step - loss: 1.1452 - accuracy: 0.7547 - val_loss: 0.9124 - val_accuracy: 0.8125
Epoch 3/5
24/24 [==============================] - 2s 79ms/step - loss: 0.8434 - accuracy: 0.7987 - val_loss: 0.7110 - val_accuracy: 0.8503
Epoch 4/5
24/24 [==============================] - 2s 77ms/step - loss: 0.6850 - accuracy: 0.8400 - val_loss: 0.6169 - val_accuracy: 0.8569
Epoch 5/5
24/24 [==============================] - 2s 78ms/step - loss: 0.5909 - accuracy: 0.8720 - val_loss: 0.5713 - val_accuracy: 0.8618


In [10]:
# Evaluate on the full test dataset
model_0.evaluate(test_data)

79/79 [==============================] - 4s 42ms/step - loss: 0.5683 - accuracy: 0.8616


[0.5682950019836426, 0.8615999817848206]

In [ ]:
# Check the layers in our base model
for layer_number, layer in enumerate(base_model.layers):
    print(layer_number, layer.name)